# [IDPP CLEF Challlenge](http://brainteaser.dei.unipd.it/challenges/idpp2023/)


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, XGBRegressor
import lightgbm

# machine learning
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import r_regression
from sklearn import preprocessing

In [2]:
DATASET_DIR = "../data/datasetA_train"
ID_FEAT = "patient_id"

pd.set_option('display.max_rows', None)

# Load and Merge data

In [88]:
def filenames_in_folder(dir_path):
    file_names = []
    for _, __, f_names in os.walk(dir_path):
        for file_name in f_names:
            file_names.append(file_name)
        break
    return file_names

def read_dfs(dir_path):
    file_names = filenames_in_folder(dir_path)
    dfs = {file_name: pd.read_csv(os.path.join(dir_path, file_name)) for file_name in file_names if file_name.endswith("csv")}
    return dfs


def merge_dfs(dfs):
    
    def merge_df_time_series(orig_df, id_feat, other_feat):
        values = orig_df.groupby(id_feat)[other_feat].apply(lambda x: x.values)
        orig_df["datasetA_train-relapses"] = pd.Series(orig_df.groupby(id_feat)[other_feat].apply(lambda x: x.values).values.tolist(), dtype=object)
        return orig_df
                        
    relapses_df = dfs["datasetA_train-relapses.csv"]
    ts_feats = ["delta_relapse_time0"]
    oo_feats = ["centre"]
                          
    test_df = merge_df_time_series(relapses_df, ID_FEAT, "delta_relapse_time0")

    merged_df = pd.merge(dfs["datasetA_train-static-vars.csv"], dfs["datasetA_train-outcomes.csv"],
                         on="patient_id", how="outer")
    
dfs = read_dfs(DATASET_DIR)
merge_dfs(dfs)
    




In [89]:
dfs = read_dfs(DATASET_DIR)



In [14]:
merged_df = pd.merge(dfs["datasetA_train-static-vars.csv"], dfs["datasetA_train-outcomes.csv"],
                         on="patient_id", how="outer")



In [97]:
def merge_dfs_complete(dfs):
    def change_ts_column_to_list(orig_df, id_feat, other_feature):
        print(orig_df.columns.to_series().groupby(orig_df.dtypes).groups)
        orig_df[other_feature] = pd.Series(orig_df.groupby(id_feat)[other_feature].apply(lambda x: x.values).values.tolist())
        return orig_df
    
    def change_ts_column_same_values_to_one(orig_df, id_feat, other_feature):
        orig_df[other_feature] = orig_df.groupby(id_feat)[other_feature].agg(pd.Series.mode)
        return orig_df

    def transpose_df_by_uniques(orig_df, id_feat, time_series_feats, one_occurrence_feats):
        for ts_feat in time_series_feats:
            orig_df = change_ts_column_to_list(orig_df, id_feat, ts_feat)
        
        # for oo_feat in one_occurrence_feats:
        #     orig_df = change_ts_column_same_values_to_one(orig_df, id_feat, ts_feat)
#         ts_dfs = [change_ts_column_to_list(orig_df, id_feat, ts_feat) for ts_feat in time_series_feats]
#         oo_dfs = [change_ts_column_same_values_to_one(orig_df, id_feat, oo_feat) for oo_feat in one_occurrence_feats]

#         out_df = pd.concat([*oo_dfs, *ts_dfs], axis=1)
        # out_df.reset_index(names=id_feat, inplace=True)
        return orig_df
    
    merged_df = pd.merge(dfs["datasetA_train-static-vars.csv"], dfs["datasetA_train-outcomes.csv"],
                         on="patient_id", how="outer")

    relapses_df = dfs["datasetA_train-relapses.csv"]
    ts_feats = ["delta_relapse_time0"]
    oo_feats = []
    relapses_df = relapses_df.drop(["centre"], axis=1)
    relapses_df = transpose_df_by_uniques(relapses_df, ID_FEAT, ts_feats, oo_feats)

#     ms_type_df = dfs["datasetA_train-ms-type.csv"]
#     ts_feats = ["multiple_sclerosis_type", "delta_observation_time0"]
#     oo_feats = ["centre"]
#     ms_type_df = transpose_df_by_uniques(ms_type_df, ID_FEAT, ts_feats, oo_feats)

#     mri_df = dfs["datasetA_train-mri.csv"]
#     ts_feats = ["mri_area_label", "lesions_T1", "lesions_T1_gadolinium", "number_of_lesions_T1_gadolinium",
#                 "new_or_enlarged_lesions_T2", "number_of_new_or_enlarged_lesions_T2", "lesions_T2", "number_of_total_lesions_T2", "delta_mri_time0"]
#     oo_feats = ["centre"]
#     mri_df = transpose_df_by_uniques(mri_df, ID_FEAT, ts_feats, oo_feats)

#     evoked_p_df = dfs["datasetA_train-evoked-potentials.csv"]
#     ts_feats = ["altered_potential", "potential_value", "location", "delta_evoked_potential_time0"]
#     oo_feats = ["centre"]
#     evoked_p_df = transpose_df_by_uniques(evoked_p_df, ID_FEAT, ts_feats, oo_feats)


#     edss_df = dfs["datasetA_train-edss.csv"]
#     ts_feats = ["edss_as_evaluated_by_clinician", "delta_edss_time0"]
#     oo_feats = ["centre"]
#     edss_df = transpose_df_by_uniques(edss_df, ID_FEAT, ts_feats, oo_feats)

    transposed_dfs = [relapses_df]
    print(relapses_df.columns.to_series().groupby(relapses_df.dtypes).groups)
    print(merged_df.columns.to_series().groupby(merged_df.dtypes).groups)
    merged_df = pd.merge(merged_df, relapses_df, on=ID_FEAT, how="inner")

    # merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()].copy()  # Removes duplicate patient_id, centre
    return merged_df

dfs = read_dfs(DATASET_DIR)
merge_dfs_complete(dfs)


{int64: ['delta_relapse_time0'], object: ['patient_id']}
{object: ['patient_id', 'delta_relapse_time0']}
{bool: ['ms_in_pediatric_age', 'spinal_cord_symptom', 'brainstem_symptom', 'eye_symptom', 'supratentorial_symptom'], int64: ['age_at_onset', 'time_since_onset', 'outcome_occurred'], float64: ['diagnostic_delay', 'outcome_time'], object: ['patient_id', 'sex', 'residence_classification', 'ethnicity', 'other_symptoms', 'centre']}


,patient_id,sex,residence_classification,ethnicity,ms_in_pediatric_age,age_at_onset,diagnostic_delay,spinal_cord_symptom,brainstem_symptom,eye_symptom,supratentorial_symptom,other_symptoms,centre,time_since_onset,outcome_occurred,outcome_time,delta_relapse_time0
0,100619256189067386770484450960632124211,female,Towns,Caucasian,False,26,18.0,False,False,True,False,False,pavia,835,1,9.400000,[-835]
1,103223150270392058352370339153314674792,male,Towns,Caucasian,False,22,115.0,True,False,False,True,False,pavia,891,0,2.057534,[-764]
2,103223150270392058352370339153314674792,male,Towns,Caucasian,False,22,115.0,True,False,False,True,False,pavia,891,0,2.057534,"[-446, -436]"
3,106924426526129864529638590750432464527,female,Rural Area,Caucasian,False,26,443.0,False,True,False,True,False,pavia,1264,0,6.958904,"[-820, -891]"
4,107682552341634765186493241028546130207,male,Towns,Caucasian,False,31,2110.0,False,True,False,False,False,pavia,2902,0,4.517808,[-876]
5,108780183989530168227228890722776788376,female,Rural Area,Caucasian,False,28,488.0,False,False,True,False,False,pavia,874,0,2.589041,[-821]
6,108780183989530168227228890722776788376,female,Rural Area,Caucasian,False,28,488.0,False,False,True,False,False,pavia,874,0,2.589041,[-833]
7,112596432759325588671631481928263599424,female,Towns,Caucasian,True,17,690.0,False,False,True,False,False,pavia,1682,0,2.054795,"[-386, -874]"
8,11359520523667992364248689835884734231,male,Cities,Caucasian,False,22,1.0,False,False,True,False,False,pavia,469,0,9.934247,[-900]
9,11359520523667992364248689835884734231,male,Cities,Caucasian,False,22,1.0,False,False,True,False,False,pavia,469,0,9.934247,"[-224, -469]"
